In [1]:
import os
import gzip, json, time
import torch 
import numpy as np
from pathlib import Path
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer, 
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig, TaskType, PeftModel, PeftConfig
from bert_score import score
import onnxruntime as ort
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import onnx
import onnxruntime as ort
from tqdm import tqdm
import re
from bert_score import score as bert_score


In [24]:
torch.cuda.empty_cache()
import gc
gc.collect()

1332

In [3]:
model_name = "codellama/CodeLlama-7b-Instruct-hf"

In [4]:
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

In [6]:
def get_model_size(model):
    total_params = sum(p.numel() for p in model.parameters())
    dtype_size = torch.finfo(model.dtype).bits // 8
    
    model_size_gb = total_params * dtype_size / (1024 ** 3)
    # print(f"Model size: {model_size_gb:2f} GB")
    return model_size_gb

In [7]:
get_model_size(model)

25.103042602539062

### Floating Point 16 params

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-Instruct-hf",
    torch_dtype=torch.float16,
    # device_map={"": 0},
    # low_cpu_mem_usage=True
    device_map="cuda:0"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
get_model_size(model)

12.551521301269531

In [27]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sun May 11 23:17:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A30                     Off |   00000000:99:00.0 Off |                    0 |
| N/A   40C    P0             34W /  165W |   23321MiB /  24576MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [11]:
data_path = "/mnt/object_group/data/"

In [12]:
train = "/mnt/object_group/data/processed/train.jsonl.gz"
with gzip.open(train, 'rt', encoding='utf-8') as f:
    for i, line in enumerate(f):
        print("    " + line.strip())
        if i >= 5:
            break


    {"comment_id": 74291026, "comment_user_login": "vikerman", "comment_body": "Is there a better way to avoid having this in different places?\n", "comment_created_at": "2016-08-10T17:30:42+00:00", "comment_html_url": "https://github.com/angular/angular/pull/10620#discussion_r74291026", "comment_path": "modules/@angular/common/src/forms-deprecated/directives/abstract_control_directive.ts", "comment_position": 10, "comment_original_position": 10, "comment_commit_id": "ee8e802b7bc25eafbe54109b3924e9dbc36dce11", "comment_original_commit_id": "ee8e802b7bc25eafbe54109b3924e9dbc36dce11", "diff_line_content": "}\n", "diff_line_type": "added", "diff_line_source_no": null, "diff_line_target_no": 15, "diff_hunk_header": "", "diff": "@@ -6,10 +6,13 @@\n  * found in the LICENSE file at https://angular.io/license\n  */\n \n-import {unimplemented} from '../../facade/exceptions';\n+import {BaseException} from '@angular/core';\n import {isPresent} from '../../facade/lang';\n import {AbstractControl} 

In [13]:
DATA_ROOT = Path("/mnt/object_group/data/processed")  # Adjust as needed

def load_samples(limit_per_split=10):
    diff_samples = []
    for repo_dir in DATDATA_ROOT = Path("/mnt/object_group/data/processed")  # Adjust as needed

def load_samples(limit_per_split=10):
    diff_samples = []
    for repo_dir in DATA_ROOT.iterdir():
        if not repo_dir.is_dir():
            continue
        for pr_dir in (repo_dir / "diff").iterdir():
            pr_id = pr_dir.name
            diff_file = pr_dir
            comments_file = DATA_ROOT / repo_dir.name / "comments" / f"{repo_dir.name}_{pr_id}_comments.jsonl"

            if not diff_file.exists() or not comments_file.exists():
                continue

            with open(diff_file, "r", encoding="utf-8") as df:
                diff_content = df.read()

            with open(comments_file, "r", encoding="utf-8") as cf:
                for line in cf:
                    try:
                        comment = json.loads(line)
                        offset = comment.get("original_position")
                        side = comment.get("side", "RIGHT")
                        body = comment.get("body", "").strip()
                        if offset is not None and body:
                            sample = {
                                "input": f"<DIFF>\n{diff_content}\n</DIFF>\n<COMMENT side=\"{side}\" offset=\"{offset}\">",
                                "output": body
                            }
                            diff_samples.append(sample)
                    except json.JSONDecodeError:
                        continue

    random.shuffle(diff_samples)
    return {
        "train": diff_samples[:limit_per_split],
        "test": diff_samples[limit_per_split:2*limit_per_split],
        "eval": diff_samples[2*limit_per_split:3*limit_per_split],
    }
A_ROOT.iterdir():
        if not repo_dir.is_dir():
            continue
        for pr_dir in (repo_dir / "diff").iterdir():
            pr_id = pr_dir.name
            diff_file = pr_dir
            comments_file = DATA_ROOT / repo_dir.name / "comments" / f"{repo_dir.name}_{pr_id}_comments.jsonl"

            if not diff_file.exists() or not comments_file.exists():
                continue

            with open(diff_file, "r", encoding="utf-8") as df:
                diff_content = df.read()

            with open(comments_file, "r", encoding="utf-8") as cf:
                for line in cf:
                    try:
                        comment = json.loads(line)
                        offset = comment.get("original_position")
                        side = comment.get("side", "RIGHT")
                        body = comment.get("body", "").strip()
                        if offset is not None and body:
                            sample = {
                                "input": f"<DIFF>\n{diff_content}\n</DIFF>\n<COMMENT side=\"{side}\" offset=\"{offset}\">",
                                "output": body
                            }
                            diff_samples.append(sample)
                    except json.JSONDecodeError:
                        continue

    random.shuffle(diff_samples)
    return {
        "train": diff_samples[:limit_per_split],
        "test": diff_samples[limit_per_split:2*limit_per_split],
        "eval": diff_samples[2*limit_per_split:3*limit_per_split],
    }


In [14]:
def load_jsonl_gz(file_path, limit):
    samples = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= limit:
                break
            entry = json.loads(line)
            prompt = f"<DIFF>\n{entry['diff']}\n</DIFF>\n"
            comment = f"<COMMENT side=\"{entry['side']}\" offset=\"{entry['line_offset']}\">{entry['comment_body']}"
            samples.append({'input': prompt, 'label': comment})
    return samples

train_data = load_jsonl_gz("/mnt/object_group/data/processed/train.jsonl.gz", 10)
test_data  = load_jsonl_gz("/mnt/object_group/data/processed/test.jsonl.gz", 8)
eval_data  = load_jsonl_gz("/mnt/object_group/data/processed/val.jsonl.gz", 4)

In [30]:
tokenizer.save_pretrained("./models/llama_onnx")
# tokenizer.save_pretrained("mnt/object/llama_onnx")

('./models/llama_onnx/tokenizer_config.json',
 './models/llama_onnx/special_tokens_map.json',
 './models/llama_onnx/tokenizer.json')

In [15]:
print(train_data[0].keys())

for i in range(3):
    print(train_data[i]["input"])
    print(train_data[i]["label"])
# print(train_data[0]["input"])

dict_keys(['input', 'label'])
<DIFF>
@@ -6,10 +6,13 @@
  * found in the LICENSE file at https://angular.io/license
  */
 
-import {unimplemented} from '../../facade/exceptions';
+import {BaseException} from '@angular/core';
 import {isPresent} from '../../facade/lang';
 import {AbstractControl} from '../model';
 
+function unimplemented(): any {
+  throw new BaseException('unimplemented');
</DIFF>

<COMMENT side="RIGHT" offset="9">Is there a better way to avoid having this in different places?

<DIFF>
@@ -6,10 +6,13 @@
  * found in the LICENSE file at https://angular.io/license
  */
 
-import {unimplemented} from '../../facade/exceptions';
+import {BaseException} from '@angular/core';
 import {isPresent} from '../../facade/lang';
 import {AbstractControl} from '../model';
 
+function unimplemented(): any {
+  throw new BaseException('unimplemented');
</DIFF>

<COMMENT side="RIGHT" offset="9">we could export it in the public API but that's not nice.
Or we could inline the method and del

In [16]:
# Prompt template
instruction = (
    "You are a helpful code reviewer. "
    "Given a code diff, generate all relevant review comments. "
    "Each comment must be in the format: "
    "<COMMENT side=\"RIGHT\" offset=\"X\">Your comment here.\n"
    "Only output comments, nothing else.\n\n"
    "### Code Diff:\n"
)

In [17]:
def extract_comments(text):
    pattern = re.compile(r'<COMMENT side="(?P<side>[^"]+)" offset="(?P<offset>\d+)">(?P<comment>.+)')
    results = []
    for line in text.strip().splitlines():
        match = pattern.match(line.strip())
        if match:
            results.append({
                "side": match.group("side"),
                "offset": int(match.group("offset")),
                "comment": match.group("comment").strip()
            })
    return results

In [44]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

In [18]:
def offset_match(pred_comments, true_comments, window=3):
    correct = 0
    total = len(pred_comments)

    gt_pairs = {(c["side"], c["offset"]) for c in true_comments}
    matched = set()

    for pred in pred_comments:
        pred_side, pred_offset = pred["side"], pred["offset"]
        for gt in gt_pairs:
            gt_side, gt_offset = gt
            if pred_side == gt_side and abs(pred_offset - gt_offset) <= window:
                matched.add(gt)
                correct += 1
                break  # count each pred only once

    return correct, total

In [20]:
references = []
candidates = []
offset_match_total = 0
offset_match_correct = 0
latencies = []

for i, sample in enumerate(tqdm(test_data[:100])):  # Full test set if needed
    prompt = tokenizer.apply_chat_template(
        [{"role": "user", "content": instruction + sample["input"]}],
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=4096
    ).to("cuda:0")

    true_comments = extract_comments(sample["label"])

    # Measure inference latency
    with torch.no_grad():
        start = time.time()
        output_ids = model.generate(**inputs, max_new_tokens=256, do_sample=False)
        latencies.append(time.time() - start)

    # Decode and evaluate
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
    generated_text = generated_text.split("Setting `pad_token_id")[0].strip()
    print(f"\n=== Sample {i} ===\n{generated_text}")

    pred_comments = extract_comments(generated_text)

    for gt in true_comments:
        references.append(gt["comment"])
        best_pred = max(pred_comments, key=lambda p: p["comment"], default={"comment": ""})
        candidates.append(best_pred["comment"] if best_pred else "")

    correct, total = offset_match(pred_comments, true_comments, window=3)
    offset_match_correct += correct
    offset_match_total += total

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Sample 0 ===
[INST] You are a helpful code reviewer. Given a code diff, generate all relevant review comments. Each comment must be in the format: <COMMENT side="RIGHT" offset="X">Your comment here.
Only output comments, nothing else.

### Code Diff:
<DIFF>
@@ -838,45 +839,58 @@ function performConcurrentWorkOnRoot(root, didTimeout) {
       throw fatalError;
     }
 
-    // Check if this render may have yielded to a concurrent event, and if so,
-    // confirm that any newly rendered stores are consistent.
-    // TODO: It's possible that even a concurrent render may never have yielded
-    // to the main thread, if it was fast enough, or if it expired. We could
-    // skip the consistency check in that case, too.
-    const renderWasConcurrent = !includesBlockingLane(root, lanes);
-    const finishedWork: Fiber = (root.current.alternate: any);
-    if (
-      renderWasConcurrent &&
-      !isRenderConsistentWithExternalStores(finishedWork)
-    ) {
-      // A store was mutat

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Sample 1 ===
[INST] You are a helpful code reviewer. Given a code diff, generate all relevant review comments. Each comment must be in the format: <COMMENT side="RIGHT" offset="X">Your comment here.
Only output comments, nothing else.

### Code Diff:
<DIFF>
@@ -838,45 +839,58 @@ function performConcurrentWorkOnRoot(root, didTimeout) {
       throw fatalError;
     }
 
-    // Check if this render may have yielded to a concurrent event, and if so,
-    // confirm that any newly rendered stores are consistent.
-    // TODO: It's possible that even a concurrent render may never have yielded
-    // to the main thread, if it was fast enough, or if it expired. We could
-    // skip the consistency check in that case, too.
-    const renderWasConcurrent = !includesBlockingLane(root, lanes);
-    const finishedWork: Fiber = (root.current.alternate: any);
-    if (
-      renderWasConcurrent &&
-      !isRenderConsistentWithExternalStores(finishedWork)
-    ) {
-      // A store was mutat

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


=== Sample 5 ===
[INST] You are a helpful code reviewer. Given a code diff, generate all relevant review comments. Each comment must be in the format: <COMMENT side="RIGHT" offset="X">Your comment here.
Only output comments, nothing else.

### Code Diff:
<DIFF>
@@ -739,14 +739,20 @@ describe('ReactCache', () => {
     await act(async () => {
       refresh();
     });
-    expect(Scheduler).toHaveYielded(['Cache miss! [A]', 'Loading...']);
+    expect(Scheduler).toHaveYielded([
+      'Cache miss! [A]',
+      'Loading...',
+      // TODO: This happens too early, because we don't retain the refreshed
+      // cache until it commits. Will fix in next step.
+      'Cache cleanup: A [v1]',
</DIFF> [/INST]  <COMMENT side="RIGHT" offset="10">This comment is suggesting that the cache cleanup is happening too early, before the refreshed cache has committed. This could be fixed by waiting for the cache to commit before cleaning up.</COMMENT>

<COMMENT side="RIGHT" offset="15">This comment is

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Sample 6 ===
[INST] You are a helpful code reviewer. Given a code diff, generate all relevant review comments. Each comment must be in the format: <COMMENT side="RIGHT" offset="X">Your comment here.
Only output comments, nothing else.

### Code Diff:
<DIFF>
@@ -0,0 +1,197 @@
+/**
+ * Copyright (c) Facebook, Inc. and its affiliates.
+ *
+ * This source code is licensed under the MIT license found in the
+ * LICENSE file in the root directory of this source tree.
+ *
+ * @emails react-core
+ */
+
+let JSDOM;
+let React;
+let ReactDOM;
+let Scheduler;
+let clientAct;
+let ReactDOMFizzServer;
+let Stream;
+let document;
+let writable;
+let container;
+let buffer = '';
+let hasErrored = false;
+let fatalError = undefined;
+let textCache;
+
+describe('useId', () => {
+  beforeEach(() => {
+    jest.resetModules();
+    JSDOM = require('jsdom').JSDOM;
+    React = require('react');
+    ReactDOM = require('react-dom');
+    Scheduler = require('scheduler');
+    clientAct = require('jest

100%|██████████| 8/8 [01:28<00:00, 11.03s/it]


=== Sample 7 ===
[INST] You are a helpful code reviewer. Given a code diff, generate all relevant review comments. Each comment must be in the format: <COMMENT side="RIGHT" offset="X">Your comment here.
Only output comments, nothing else.

### Code Diff:
<DIFF>
@@ -389,17 +389,6 @@ describe('ReactSuspense', () => {
     expect(root).toMatchRenderedOutput('Hi');
   });
 
-  it('throws if tree suspends and none of the Suspense ancestors have a boundary', () => {
</DIFF> [/INST]  <COMMENT side="RIGHT" offset="389">This test seems to be testing the behavior of the `ReactSuspense` component when it is used in a situation where it is not expected to be used. The test is checking that an error is thrown when the component suspends and none of the Suspense ancestors have a boundary. However, this test is not actually testing the behavior of the `ReactSuspense` component in this situation. It would be more useful to test the behavior of the component in a real-world scenario where it is actual

In [21]:
# Latency stats
print("\n=== Inference Latency Stats ===")
print(f"Median latency:        {np.percentile(latencies, 50) * 1000:.2f} ms")
print(f"95th percentile:       {np.percentile(latencies, 95) * 1000:.2f} ms")
print(f"99th percentile:       {np.percentile(latencies, 99) * 1000:.2f} ms")
print(f"Throughput (1 sample): {len(latencies) / np.sum(latencies):.2f} samples/sec")


=== Inference Latency Stats ===
Median latency:        11944.54 ms
95th percentile:       11986.58 ms
99th percentile:       11990.94 ms
Throughput (1 sample): 0.09 samples/sec


In [22]:
# BERTScore
P, R, F1 = bert_score(candidates, references, lang="en", verbose=True)
print("\nEvaluation Results")
print(f"Avg BERTScore F1: {F1.mean().item():.4f}")
print(f"Offset Precision: {offset_match_correct}/{offset_match_total} = {offset_match_correct/offset_match_total:.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.15 seconds, 55.11 sentences/sec

Evaluation Results
Avg BERTScore F1: 0.7275
Offset Precision: 0/23 = 0.0000


Batch throughput test

In [23]:
batch_size = 4
num_batches = 10
batch_inputs = [test_data[i]["input"] for i in range(batch_size)]
batch_prompts = [
    tokenizer.apply_chat_template(
        [{"role": "user", "content": instruction + inp}],
        tokenize=False,
        add_generation_prompt=True
    )
    for inp in batch_inputs
]

encoded = tokenizer(
    batch_prompts,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=4096
).to("cuda:0")

# Warm-up
with torch.no_grad():
    _ = model.generate(**encoded, max_new_tokens=256, do_sample=False)

batch_times = []
with torch.no_grad():
    for _ in range(num_batches):
        start = time.time()
        _ = model.generate(**encoded, max_new_tokens=256, do_sample=False)
        batch_times.append(time.time() - start)

batch_fps = (batch_size * num_batches) / np.sum(batch_times)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 344.00 MiB. GPU 0 has a total capacity of 23.60 GiB of which 158.25 MiB is free. Process 24736 has 23.44 GiB memory in use. Of the allocated memory 20.11 GiB is allocated by PyTorch, and 3.05 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print("\n=== Final Evaluation Summary ===")
print(f"Model Size on Disk:               {get_model_size(model):.2f}")
print(f"BERTScore F1 (avg):               {F1.mean().item():.4f}")
print(f"Location Precision:               {offset_match_correct}/{offset_match_total} = {offset_match_correct / offset_match_total:.4f}")
print(f"Inference Latency (median):       {np.percentile(latencies, 50) * 1000:.2f} ms")
print(f"Inference Latency (95th pct):     {np.percentile(latencies, 95) * 1000:.2f} ms")
print(f"Inference Latency (99th pct):     {np.percentile(latencies, 99) * 1000:.2f} ms")
print(f"Inference Throughput (1 sample):  {len(latencies) / np.sum(latencies):.2f} samples/sec")
print(f"Batch Throughput ({batch_size}):  {batch_fps:.2f} samples/sec")


In [ ]:
!nvidia-smi

### Compiled model

In [42]:
model.compile()

In [48]:
references = []
candidates = []
offset_match_total = 0
offset_match_correct = 0

for i, sample in enumerate(tqdm(test_data[:5])):  # Full test set if needed
    prompt = tokenizer.apply_chat_template(
        [{"role": "user", "content": instruction + sample["input"]}],
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=4096
    ).to("cuda:0")

    true_comments = extract_comments(sample["label"])

    # Measure inference latency
    with torch.no_grad():
        start = time.time()
        output_ids = model.generate(**inputs, max_new_tokens=256, do_sample=False)
        latencies.append(time.time() - start)

    # Decode and evaluate
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
    generated_text = generated_text.split("Setting `pad_token_id")[0].strip()
    print(f"\n=== Sample {i} ===\n{generated_text}")

    pred_comments = extract_comments(generated_text)

    for gt in true_comments:
        references.append(gt["comment"])
        best_pred = max(pred_comments, key=lambda p: p["comment"], default={"comment": ""})
        candidates.append(best_pred["comment"] if best_pred else "")

    correct, total = offset_match(pred_comments, true_comments, window=3)
    offset_match_correct += correct
    offset_match_total += total

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Sample 0 ===
[INST] You are a helpful code reviewer. Given a code diff, generate all relevant review comments. Each comment must be in the format: <COMMENT side="RIGHT" offset="X">Your comment here.
Only output comments, nothing else.

### Code Diff:
<DIFF>
@@ -838,45 +839,58 @@ function performConcurrentWorkOnRoot(root, didTimeout) {
       throw fatalError;
     }
 
-    // Check if this render may have yielded to a concurrent event, and if so,
-    // confirm that any newly rendered stores are consistent.
-    // TODO: It's possible that even a concurrent render may never have yielded
-    // to the main thread, if it was fast enough, or if it expired. We could
-    // skip the consistency check in that case, too.
-    const renderWasConcurrent = !includesBlockingLane(root, lanes);
-    const finishedWork: Fiber = (root.current.alternate: any);
-    if (
-      renderWasConcurrent &&
-      !isRenderConsistentWithExternalStores(finishedWork)
-    ) {
-      // A store was mutat

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Sample 1 ===
[INST] You are a helpful code reviewer. Given a code diff, generate all relevant review comments. Each comment must be in the format: <COMMENT side="RIGHT" offset="X">Your comment here.
Only output comments, nothing else.

### Code Diff:
<DIFF>
@@ -838,45 +839,58 @@ function performConcurrentWorkOnRoot(root, didTimeout) {
       throw fatalError;
     }
 
-    // Check if this render may have yielded to a concurrent event, and if so,
-    // confirm that any newly rendered stores are consistent.
-    // TODO: It's possible that even a concurrent render may never have yielded
-    // to the main thread, if it was fast enough, or if it expired. We could
-    // skip the consistency check in that case, too.
-    const renderWasConcurrent = !includesBlockingLane(root, lanes);
-    const finishedWork: Fiber = (root.current.alternate: any);
-    if (
-      renderWasConcurrent &&
-      !isRenderConsistentWithExternalStores(finishedWork)
-    ) {
-      // A store was mutat

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Sample 2 ===
[INST] You are a helpful code reviewer. Given a code diff, generate all relevant review comments. Each comment must be in the format: <COMMENT side="RIGHT" offset="X">Your comment here.
Only output comments, nothing else.

### Code Diff:
<DIFF>
@@ -1149,6 +1163,10 @@ function performSyncWorkOnRoot(root) {
     throw fatalError;
   }
 
+  if (exitStatus === RootDidNotComplete) {
</DIFF> [/INST]  <COMMENT side="RIGHT" offset="1150">This line is not needed. The `throw` statement will already cause the function to exit with an error. It's not necessary to check the `exitStatus` variable and throw a new error if it's not `RootCompleted`.</COMMENT>

<COMMENT side="RIGHT" offset="1164">This line is not needed. The `if` statement is already checking if the `exitStatus` variable is equal to `RootDidNotComplete`. It's not necessary to check the `exitStatus` variable again in the `if` statement's body.</COMMENT>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Sample 3 ===
[INST] You are a helpful code reviewer. Given a code diff, generate all relevant review comments. Each comment must be in the format: <COMMENT side="RIGHT" offset="X">Your comment here.
Only output comments, nothing else.

### Code Diff:
<DIFF>
@@ -739,14 +739,20 @@ describe('ReactCache', () => {
     await act(async () => {
       refresh();
     });
-    expect(Scheduler).toHaveYielded(['Cache miss! [A]', 'Loading...']);
+    expect(Scheduler).toHaveYielded([
+      'Cache miss! [A]',
+      'Loading...',
+      // TODO: This happens too early, because we don't retain the refreshed
+      // cache until it commits. Will fix in next step.
+      'Cache cleanup: A [v1]',
</DIFF> [/INST]  <COMMENT side="RIGHT" offset="10">This comment is suggesting that the cache cleanup is happening too early, before the refreshed cache has committed. This could be fixed by waiting for the cache to commit before cleaning up.</COMMENT>

<COMMENT side="RIGHT" offset="15">This comment is

100%|██████████| 5/5 [00:35<00:00,  7.14s/it]


=== Sample 4 ===
[INST] You are a helpful code reviewer. Given a code diff, generate all relevant review comments. Each comment must be in the format: <COMMENT side="RIGHT" offset="X">Your comment here.
Only output comments, nothing else.

### Code Diff:
<DIFF>
@@ -739,14 +739,20 @@ describe('ReactCache', () => {
     await act(async () => {
       refresh();
     });
-    expect(Scheduler).toHaveYielded(['Cache miss! [A]', 'Loading...']);
+    expect(Scheduler).toHaveYielded([
+      'Cache miss! [A]',
+      'Loading...',
+      // TODO: This happens too early, because we don't retain the refreshed
+      // cache until it commits. Will fix in next step.
+      'Cache cleanup: A [v1]',
</DIFF> [/INST]  <COMMENT side="RIGHT" offset="10">This comment is suggesting that the cache cleanup is happening too early, before the refreshed cache has committed. This could be fixed by waiting for the cache to commit before cleaning up.</COMMENT>

<COMMENT side="RIGHT" offset="15">This comment is

In [49]:
batch_size = 4
num_batches = 10
batch_inputs = [test_data[i]["input"] for i in range(batch_size)]
batch_prompts = [
    tokenizer.apply_chat_template(
        [{"role": "user", "content": instruction + inp}],
        tokenize=False,
        add_generation_prompt=True
    )
    for inp in batch_inputs
]

encoded = tokenizer(
    batch_prompts,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=4096
).to("cuda:0")

# Warm-up
with torch.no_grad():
    _ = model.generate(**encoded, max_new_tokens=256, do_sample=False)

batch_times = []
with torch.no_grad():
    for _ in range(num_batches):
        start = time.time()
        _ = model.generate(**encoded, max_new_tokens=256, do_sample=False)
        batch_times.append(time.time() - start)

batch_fps = (batch_size * num_batches) / np.sum(batch_times)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [50]:
print("\n=== Final Evaluation Summary ===")
print(f"Model Size on Disk:               {get_model_size(model):.2f}")
print(f"BERTScore F1 (avg):               {F1.mean().item():.4f}")
print(f"Location Precision:               {offset_match_correct}/{offset_match_total} = {offset_match_correct / offset_match_total:.4f}")
print(f"Inference Latency (median):       {np.percentile(latencies, 50) * 1000:.2f} ms")
print(f"Inference Latency (95th pct):     {np.percentile(latencies, 95) * 1000:.2f} ms")
print(f"Inference Latency (99th pct):     {np.percentile(latencies, 99) * 1000:.2f} ms")
print(f"Inference Throughput (1 sample):  {len(latencies) / np.sum(latencies):.2f} samples/sec")
print(f"Batch Throughput ({batch_size}):  {batch_fps:.2f} samples/sec")



=== Final Evaluation Summary ===
Model Size on Disk:               12.55
BERTScore F1 (avg):               0.7252
Location Precision:               0/16 = 0.0000
Inference Latency (median):       7761.92 ms
Inference Latency (95th pct):     8123.67 ms
Inference Latency (99th pct):     159196.10 ms
Inference Throughput (1 sample):  0.06 samples/sec
Batch Throughput (4):  0.26 samples/sec


### Converting to ONNX

The newer versions don't support Optimum, and with ORTModelForCausalLM, it exports the onnx model too

In [24]:
model_id = "codellama/CodeLLaMA-7b-Instruct-hf"  
onnx_dir = "models/llama_onnx/"

model = ORTModelForCausalLM.from_pretrained(
    model_id, 
    export=True, 
    cache_dir=onnx_dir
)

model.save_pretrained(onnx_dir)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.save_pretrained(onnx_dir)

print(f"ONNX model saved to: {onnx_dir}/model.onnx")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/transformers/modeling_attn_mask_utils.py:114: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (input_shape[-1] > 1 or self.sliding_window is not None) and self.is_causal:
/opt/conda/lib/python3.12/site-packages/optimum/exporters/onnx/model_patcher.py:454: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future

ONNX model saved to: models/llama_onnx//model.onnx


In [6]:
# !pip uninstall onnxruntime -y
# !pip install onnxruntime-gpu
print(ort.get_available_providers())


AttributeError: module 'onnxruntime' has no attribute 'get_available_providers'

In [5]:
ort_session = ort.InferenceSession("models/llama_onnx/model.onnx", providers=["CUDAExecutionProvider"])

AttributeError: module 'onnxruntime' has no attribute 'InferenceSession'

In [36]:
# ort_session.get_providers()
import onnxruntime as ort
print(ort.get_available_providers())

['AzureExecutionProvider', 'CPUExecutionProvider']


In [41]:
!nvidia-smi

Sun May 11 13:39:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000000:27:00.0 Off |                    0 |
| N/A   45C    P0             48W /  300W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [38]:
!python -c "import onnxruntime as ort; print(ort.get_available_providers())"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
AttributeError: module 'onnxruntime' has no attribute 'get_available_providers'


In [26]:
onnx_dir = "models/llama_onnx/"

# Load the ONNX model
model = ORTModelForCausalLM.from_pretrained(onnx_dir).to("cuda")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(onnx_dir)

ValueError: Asked to use CUDAExecutionProvider as an ONNX Runtime execution provider, but the available execution providers are ['AzureExecutionProvider', 'CPUExecutionProvider'].

In [ ]:
code_prompt = """
def fibonacci(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]
    else:
        seq = [0, 1]
        for i in range(2, n):
            seq.append(seq[-1] + seq[-2])
        return seq
"""

prompt = f"Given the code diff below, generate relevant code review comments in the format:\n<COMMENT side=\"RIGHT\" offset=\"X\">...\n\n{code_prompt}"

inputs = tokenizer(prompt, return_tensors="pt", padding="max_length", truncation=True, max_length=4096).to("cuda")

In [ ]:
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id
    )

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generated Review Comments:\n", generated_text)


In [ ]:
def benchmark_session(ort_session, input_array, batch_input):
    print(f"\nExecution provider: {ort_session.get_providers()}")
    input_name = ort_session.get_inputs()[0].name

    # Inference latency (single input)
    num_trials = 10
    ort_session.run(None, {input_name: input_array})  # Warm-up
    latencies = []
    for _ in range(num_trials):
        start = time.time()
        ort_session.run(None, {input_name: input_array})
        latencies.append(time.time() - start)

    print(f"Inference Latency (median): {np.percentile(latencies, 50) * 1000:.2f} ms")
    print(f"Inference Latency (95th pct): {np.percentile(latencies, 95) * 1000:.2f} ms")
    print(f"Inference Latency (99th pct): {np.percentile(latencies, 99) * 1000:.2f} ms")
    print(f"Inference Throughput (1 sample): {num_trials / np.sum(latencies):.2f} samples/sec")

    # Batch throughput
    num_batches = 5
    ort_session.run(None, {input_name: batch_input})  # Warm-up
    batch_times = []
    for _ in range(num_batches):
        start = time.time()
        ort_session.run(None, {input_name: batch_input})
        batch_times.append(time.time() - start)
    batch_fps = (batch_input.shape[0] * num_batches) / np.sum(batch_times)
    print(f"Batch Throughput ({batch_input.shape[0]}): {batch_fps:.2f} samples/sec")

    # Model size
    model_size = os.path.getsize(ort_session._model_path)
    print(f"Model Size on Disk: {model_size / 1e6:.2f} MB")


In [ ]:
prompt = f"Given the code diff below, generate review comments:{train_data[0]['input']}"

# Tokenize it
inputs = tokenizer(prompt, return_tensors="np", padding="max_length", truncation=True, max_length=128)

# Extract NumPy input
input_array = inputs["input_ids"]
batch_input = np.repeat(input_array, repeats=4, axis=0)  # batch of 4

In [42]:
pip show onnxruntime-gpu

Name: onnxruntime-gpu
Version: 1.22.0
Summary: ONNX Runtime is a runtime accelerator for Machine Learning models
Home-page: https://onnxruntime.ai
Author: Microsoft Corporation
Author-email: onnxruntime@microsoft.com
License: MIT License
Location: /opt/conda/lib/python3.12/site-packages
Requires: coloredlogs, flatbuffers, numpy, packaging, protobuf, sympy
Required-by: 
Note: you may need to restart the kernel to use updated packages.
